# 🎓 Data Analytics Workshop: Antioquia Higher Education Scholarships Analysis

## 📋 Workshop Overview
In this hands-on workshop, you'll analyze real data about scholarship and credit beneficiaries for higher education access programs in Antioquia, Colombia.

### 📊 Dataset Information
- **Records**: ~14,500 scholarship beneficiaries
- **Columns**: 17 variables
- **Time Period**: 2016 onwards
- **Focus**: Education access, demographics, and graduation outcomes

---
## Part 1: Data Loading and Initial Exploration 🔍

### Task 1.1: Load the Dataset

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Configure pandas display options for better readability
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Load the CSV file
file_path = 'Beneficiaros_de_becas_y_creditos_de_programas_de_acceso_a_la_educaci_n_superior_de_Antioquia_20250815.csv'
df = pd.read_csv(file_path)

# Display dataset dimensions
print(f"📊 Dataset loaded successfully!")
print(f"Dimensions: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\nExpected: ~14,566 rows × 17 columns")

### Task 1.2: Apply the "Big 3" Analysis

In [ ]:
# 1. HEAD() - Display first 5 rows
print("📋 FIRST 5 ROWS OF THE DATASET:")
print("="*80)
df.head()

In [ ]:
# 2. INFO() - General information about the dataset
print("📊 DATASET INFORMATION:")
print("="*80)
df.info()

In [ ]:
# 3. DESCRIBE() - Statistical summary
print("📈 STATISTICAL SUMMARY:")
print("="*80)
df.describe()

### Task 1.3: Column Analysis

In [ ]:
# Print all column names with their positions
print("📋 AVAILABLE COLUMNS:")
print("="*80)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\n📊 Total columns: {len(df.columns)}")

In [ ]:
# Check unique values for key categorical columns
print("🔍 UNIQUE VALUES IN KEY COLUMNS:")
print("="*80)

key_columns = ['GÉNERO', 'TIPO DE FORMACIÓN', 'GRADUADO', 'BENEFICIO OTORGADO']

for col in key_columns:
    if col in df.columns:
        unique_values = df[col].unique()
        print(f"\n📍 {col}:")
        print(f"   - Unique values: {df[col].nunique()}")
        print(f"   - Values: {unique_values[:10] if len(unique_values) > 10 else unique_values}")

In [ ]:
# Count unique universities and programs
print("🎓 EDUCATIONAL INSTITUTIONS AND PROGRAMS:")
print("="*80)
print(f"\nNumber of unique universities: {df['UNIVERSIDAD'].nunique()}")
print(f"Number of unique programs: {df['PROGRAMA CURSADO'].nunique()}")
print(f"Number of unique municipalities: {df['MUNICIPIO DE RESIDENCIA'].nunique()}")

# Top 5 universities by enrollment
print("\n🏆 Top 5 Universities by Enrollment:")
print(df['UNIVERSIDAD'].value_counts().head())

---
## Part 2: Data Cleaning and Preparation 🧹

### Task 2.1: Data Type Conversion

In [ ]:
# Check current data types
print("🔍 CURRENT DATA TYPES:")
print("="*80)
print(df.dtypes)

# Identify columns that need conversion
print("\n⚠️ Columns that might need type conversion:")
print("- ESTRATO (should be numeric)")
print("- CONVOCATORIA (should be numeric/year)")
print("- FECHA DE NACIMIENTO (should be datetime)")

In [ ]:
# Check ESTRATO values
print("🏠 ESTRATO VALUES:")
print("="*80)
print("Sample values:")
print(df['ESTRATO'].value_counts())

In [ ]:
# Convert ESTRATO to numeric
print("🔧 CONVERTING ESTRATO TO NUMERIC:")
print("="*80)

# Method 1: Extract number from string (e.g., 'ESTRATO 1' -> 1)
df['ESTRATO_NUMERIC'] = df['ESTRATO'].str.extract('(\d+)').astype(float)

# Verify conversion
print(f"Original type: {df['ESTRATO'].dtype}")
print(f"New type: {df['ESTRATO_NUMERIC'].dtype}")
print(f"\nValue counts after conversion:")
print(df['ESTRATO_NUMERIC'].value_counts().sort_index())

# Check for any conversion failures
null_count = df['ESTRATO_NUMERIC'].isnull().sum()
print(f"\nConversion failures (NaN): {null_count}")

In [ ]:
# Convert CONVOCATORIA to integer
print("📅 CONVERTING CONVOCATORIA TO NUMERIC:")
print("="*80)

# Check current values
print("Current unique values:")
print(df['CONVOCATORIA'].unique())

# Convert to numeric
df['CONVOCATORIA'] = pd.to_numeric(df['CONVOCATORIA'], errors='coerce')

print(f"\nConverted type: {df['CONVOCATORIA'].dtype}")
print(f"Year range: {df['CONVOCATORIA'].min():.0f} - {df['CONVOCATORIA'].max():.0f}")

### Task 2.2: Data Quality Assessment

In [ ]:
# Check for null values
print("🔍 MISSING DATA ANALYSIS:")
print("="*80)

# Calculate missing data
null_counts = df.isnull().sum()
null_percentages = (null_counts / len(df)) * 100

# Create summary DataFrame
missing_data = pd.DataFrame({
    'Column': null_counts.index,
    'Missing_Count': null_counts.values,
    'Percentage': null_percentages.values
})

# Filter to show only columns with missing data
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Percentage', ascending=False)

if len(missing_data) > 0:
    print(missing_data)
else:
    print("✅ No missing values found in the dataset!")

print(f"\n📊 Total missing values: {null_counts.sum()}")
print(f"📊 Percentage of complete data: {100 - (null_counts.sum() / (len(df) * len(df.columns)) * 100):.2f}%")

In [ ]:
# Verify ESTRATO values are within expected range (1-6)
print("✅ VALIDATING ESTRATO VALUES:")
print("="*80)

# Check range
estrato_min = df['ESTRATO_NUMERIC'].min()
estrato_max = df['ESTRATO_NUMERIC'].max()

print(f"ESTRATO range: {estrato_min:.0f} - {estrato_max:.0f}")
print(f"Expected range: 1 - 6")

# Check for values outside expected range
invalid_estrato = df[(df['ESTRATO_NUMERIC'] < 1) | (df['ESTRATO_NUMERIC'] > 6)]
if len(invalid_estrato) > 0:
    print(f"\n⚠️ Found {len(invalid_estrato)} records with ESTRATO outside 1-6 range")
else:
    print("\n✅ All ESTRATO values are within the expected range (1-6)")

# Distribution of ESTRATO
print("\n📊 ESTRATO Distribution:")
print(df['ESTRATO_NUMERIC'].value_counts().sort_index())

---
## Part 3: Basic Analysis with Pandas 📊

### Task 3.1: Series Operations

In [ ]:
# Extract GÉNERO column as a Series
print("👥 GENDER DISTRIBUTION ANALYSIS:")
print("="*80)

# Get gender Series
gender_series = df['GÉNERO']
print(f"Type: {type(gender_series)}")
print(f"Length: {len(gender_series)}")

# Calculate value counts
gender_counts = gender_series.value_counts()
print("\n📊 Gender Distribution:")
print(gender_counts)

# Calculate percentages
gender_percentages = gender_series.value_counts(normalize=True) * 100
print("\n📊 Gender Distribution (%)")
for gender, percentage in gender_percentages.items():
    print(f"   {gender}: {percentage:.1f}%")

In [ ]:
# Analyze GRADUADO status
print("🎓 GRADUATION STATUS ANALYSIS:")
print("="*80)

# Get graduation Series
graduation_series = df['GRADUADO']

# Calculate value counts
graduation_counts = graduation_series.value_counts()
print("\n📊 Graduation Status:")
print(graduation_counts)

# Calculate graduation rate
total_students = len(df)
graduated_students = (df['GRADUADO'] == 'SI').sum()
graduation_rate = (graduated_students / total_students) * 100

print(f"\n🎯 Overall Graduation Rate: {graduation_rate:.2f}%")
print(f"   - Graduated: {graduated_students:,} students")
print(f"   - Not graduated: {total_students - graduated_students:,} students")

### Task 3.2: DataFrame Filtering

In [ ]:
# Filter students from ESTRATO 1 only
print("🏠 FILTERING ESTRATO 1 STUDENTS:")
print("="*80)

estrato_1_students = df[df['ESTRATO_NUMERIC'] == 1]
print(f"Total students from ESTRATO 1: {len(estrato_1_students):,}")
print(f"Percentage of total: {(len(estrato_1_students) / len(df)) * 100:.1f}%")

# Show sample
print("\nSample of ESTRATO 1 students:")
estrato_1_students[['MUNICIPIO DE RESIDENCIA', 'UNIVERSIDAD', 'GRADUADO']].head()

In [ ]:
# Find all graduated students
print("🎓 FILTERING GRADUATED STUDENTS:")
print("="*80)

graduated_students = df[df['GRADUADO'] == 'SI']
print(f"Total graduated students: {len(graduated_students):,}")
print(f"Percentage of total: {(len(graduated_students) / len(df)) * 100:.1f}%")

# Analyze graduated students by education type
print("\n📊 Graduated Students by Education Type:")
print(graduated_students['TIPO DE FORMACIÓN'].value_counts())

In [ ]:
# Filter female students who graduated
print("👩‍🎓 FILTERING FEMALE GRADUATED STUDENTS:")
print("="*80)

female_graduated = df[(df['GÉNERO'] == 'FEMENINO') & (df['GRADUADO'] == 'SI')]
print(f"Female graduated students: {len(female_graduated):,}")

# Compare with male graduated students
male_graduated = df[(df['GÉNERO'] == 'MASCULINO') & (df['GRADUADO'] == 'SI')]
print(f"Male graduated students: {len(male_graduated):,}")

# Calculate graduation rates by gender
female_total = (df['GÉNERO'] == 'FEMENINO').sum()
male_total = (df['GÉNERO'] == 'MASCULINO').sum()

female_grad_rate = (len(female_graduated) / female_total) * 100
male_grad_rate = (len(male_graduated) / male_total) * 100

print(f"\n📊 Graduation Rates by Gender:")
print(f"   Female: {female_grad_rate:.2f}%")
print(f"   Male: {male_grad_rate:.2f}%")
print(f"   Gender gap: {abs(female_grad_rate - male_grad_rate):.2f} percentage points")

### Task 3.3: Using loc and iloc

In [ ]:
# Use iloc for position-based selection
print("🔢 ILOC - POSITION-BASED INDEXING:")
print("="*80)

# Select first 10 rows and columns 0-5
print("First 10 rows, columns 0-5:")
df.iloc[0:10, 0:5]

In [ ]:
# Use loc for label-based selection
print("🏷️ LOC - LABEL-BASED INDEXING:")
print("="*80)

# Select specific columns for graduated students
graduated_info = df.loc[
    df['GRADUADO'] == 'SI',
    ['UNIVERSIDAD', 'PROGRAMA CURSADO', 'TIPO DE FORMACIÓN', 'ESTRATO_NUMERIC']
]

print(f"Selected {len(graduated_info)} graduated students with specific columns")
print("\nSample of selected data:")
graduated_info.head()

In [ ]:
# Compare loc vs iloc with conditions
print("🔍 LOC WITH MULTIPLE CONDITIONS:")
print("="*80)

# Using loc with conditions
filtered_data = df.loc[
    (df['ESTRATO_NUMERIC'] <= 2) & 
    (df['TIPO DE FORMACIÓN'] == 'UNIVERSITARIA') &
    (df['GRADUADO'] == 'SI'),
    ['MUNICIPIO DE RESIDENCIA', 'UNIVERSIDAD', 'PROGRAMA CURSADO']
]

print(f"Students from ESTRATO 1-2, University level, Graduated: {len(filtered_data)}")
print("\nTop 5 programs for this group:")
print(filtered_data['PROGRAMA CURSADO'].value_counts().head())

---
## Part 4: NumPy Integration 🔢

### Task 4.1: Converting to NumPy Arrays

In [ ]:
# Convert ESTRATO column to NumPy array
print("🔢 NUMPY ARRAY OPERATIONS:")
print("="*80)

# Convert to NumPy array
estrato_array = df['ESTRATO_NUMERIC'].fillna(0).values

print(f"Type: {type(estrato_array)}")
print(f"Shape: {estrato_array.shape}")
print(f"Data type: {estrato_array.dtype}")

# Calculate statistics using NumPy
print("\n📊 NumPy Statistics:")
print(f"   Mean: {np.mean(estrato_array):.2f}")
print(f"   Median: {np.median(estrato_array):.2f}")
print(f"   Std Dev: {np.std(estrato_array):.2f}")
print(f"   Min: {np.min(estrato_array):.0f}")
print(f"   Max: {np.max(estrato_array):.0f}")

In [ ]:
# Compare speed: pandas vs NumPy
import time

print("⚡ SPEED COMPARISON: PANDAS vs NUMPY")
print("="*80)

# Create a large array for testing
test_data = df['ESTRATO_NUMERIC'].fillna(0).values

# Pandas operation
start = time.time()
for _ in range(1000):
    pandas_mean = df['ESTRATO_NUMERIC'].mean()
pandas_time = time.time() - start

# NumPy operation
start = time.time()
for _ in range(1000):
    numpy_mean = np.mean(test_data)
numpy_time = time.time() - start

print(f"Pandas time (1000 iterations): {pandas_time:.4f} seconds")
print(f"NumPy time (1000 iterations): {numpy_time:.4f} seconds")
print(f"\n🚀 NumPy is {pandas_time/numpy_time:.1f}x faster!")

### Task 4.2: Vectorized Operations

In [ ]:
# Create binary arrays for graduated status
print("🔄 VECTORIZED OPERATIONS:")
print("="*80)

# Convert graduated status to binary (1/0)
graduated_binary = np.where(df['GRADUADO'] == 'SI', 1, 0)

print(f"Binary array shape: {graduated_binary.shape}")
print(f"Sum of graduated (1s): {np.sum(graduated_binary)}")
print(f"Sum of not graduated (0s): {len(graduated_binary) - np.sum(graduated_binary)}")

# Calculate graduation rate using NumPy
graduation_rate_np = np.mean(graduated_binary) * 100
print(f"\n🎓 Graduation rate (NumPy): {graduation_rate_np:.2f}%")

In [ ]:
# Categorize ESTRATO using NumPy
print("📊 CATEGORIZING ESTRATO WITH NUMPY:")
print("="*80)

# Create categories: Low (1-2), Medium (3-4), High (5-6)
estrato_values = df['ESTRATO_NUMERIC'].fillna(0).values

# Using np.select for multiple conditions
conditions = [
    (estrato_values >= 1) & (estrato_values <= 2),
    (estrato_values >= 3) & (estrato_values <= 4),
    (estrato_values >= 5) & (estrato_values <= 6)
]
choices = ['Low', 'Medium', 'High']

estrato_categories = np.select(conditions, choices, default='Unknown')

# Add to DataFrame
df['ESTRATO_CATEGORY'] = estrato_categories

# Show distribution
print("Distribution of ESTRATO categories:")
print(df['ESTRATO_CATEGORY'].value_counts())

# Calculate graduation rates by category
print("\n🎓 Graduation rates by ESTRATO category:")
for category in ['Low', 'Medium', 'High']:
    mask = df['ESTRATO_CATEGORY'] == category
    if mask.sum() > 0:
        rate = (df[mask]['GRADUADO'] == 'SI').sum() / mask.sum() * 100
        print(f"   {category}: {rate:.2f}%")

---
## Part 5: GroupBy and Aggregations 🔬

### Task 5.1: Single Variable GroupBy

In [ ]:
# Group by GÉNERO and calculate graduation rates
print("👥 ANALYSIS BY GENDER:")
print("="*80)

# Create graduation rate calculation function
def calculate_graduation_rate(group):
    return (group == 'SI').sum() / len(group) * 100

# Group by gender
gender_analysis = df.groupby('GÉNERO').agg({
    'GRADUADO': [lambda x: (x == 'SI').sum(), 'count', calculate_graduation_rate],
    'ESTRATO_NUMERIC': 'mean'
})

# Rename columns for clarity
gender_analysis.columns = ['Graduated_Count', 'Total_Count', 'Graduation_Rate(%)', 'Avg_Estrato']
print(gender_analysis.round(2))

In [ ]:
# Group by ESTRATO and count students
print("🏠 ANALYSIS BY SOCIOECONOMIC STRATUM:")
print("="*80)

estrato_analysis = df.groupby('ESTRATO_NUMERIC').agg({
    'GRADUADO': ['count', lambda x: (x == 'SI').sum()],
    'GÉNERO': lambda x: (x == 'FEMENINO').sum()
})

# Calculate graduation rate
estrato_analysis.columns = ['Total_Students', 'Graduated', 'Female_Count']
estrato_analysis['Graduation_Rate(%)'] = (estrato_analysis['Graduated'] / estrato_analysis['Total_Students']) * 100
estrato_analysis['Female_Percentage(%)'] = (estrato_analysis['Female_Count'] / estrato_analysis['Total_Students']) * 100

print(estrato_analysis.round(2))

In [ ]:
# Group by TIPO DE FORMACIÓN
print("🎓 ANALYSIS BY EDUCATION TYPE:")
print("="*80)

education_type_analysis = df.groupby('TIPO DE FORMACIÓN').agg({
    'GRADUADO': ['count', lambda x: (x == 'SI').sum()],
    'ESTRATO_NUMERIC': 'mean'
})

education_type_analysis.columns = ['Total_Students', 'Graduated', 'Avg_Estrato']
education_type_analysis['Graduation_Rate(%)'] = (
    education_type_analysis['Graduated'] / education_type_analysis['Total_Students']
) * 100

# Sort by graduation rate
education_type_analysis = education_type_analysis.sort_values('Graduation_Rate(%)', ascending=False)
print(education_type_analysis.round(2))

In [ ]:
# Top 10 universities by student count
print("🏆 TOP 10 UNIVERSITIES BY ENROLLMENT:")
print("="*80)

university_enrollment = df.groupby('UNIVERSIDAD').size().sort_values(ascending=False).head(10)

for i, (university, count) in enumerate(university_enrollment.items(), 1):
    percentage = (count / len(df)) * 100
    print(f"{i:2d}. {university[:50]:<50} {count:>5} students ({percentage:.1f}%)")

### Task 5.2: Multiple Aggregations

In [ ]:
# Group by SUBREGIÓN with multiple metrics
print("🌍 COMPREHENSIVE ANALYSIS BY SUBREGION:")
print("="*80)

# Define aggregation functions
subregion_analysis = df.groupby('SUBREGIÓN DE RESIDENCIA').agg({
    'GRADUADO': [
        'count',
        lambda x: (x == 'SI').sum(),
        lambda x: (x == 'SI').sum() / len(x) * 100
    ],
    'ESTRATO_NUMERIC': ['mean', 'median'],
    'GÉNERO': lambda x: (x == 'FEMENINO').sum() / len(x) * 100
})

# Flatten column names
subregion_analysis.columns = [
    'Total_Students', 'Graduated_Count', 'Graduation_Rate(%)',
    'Avg_Estrato', 'Median_Estrato', 'Female_Percentage(%)'
]

# Sort by total students
subregion_analysis = subregion_analysis.sort_values('Total_Students', ascending=False)
print(subregion_analysis.round(2))

### Task 5.3: Multi-level GroupBy

In [ ]:
# Group by GÉNERO and TIPO DE FORMACIÓN
print("📊 MULTI-DIMENSIONAL ANALYSIS: GENDER × EDUCATION TYPE")
print("="*80)

# Multi-level groupby
multi_group = df.groupby(['GÉNERO', 'TIPO DE FORMACIÓN']).agg({
    'GRADUADO': [
        'count',
        lambda x: (x == 'SI').sum(),
        lambda x: (x == 'SI').sum() / len(x) * 100
    ]
})

multi_group.columns = ['Total', 'Graduated', 'Graduation_Rate(%)']
print(multi_group.round(2))

# Create pivot table for better visualization
print("\n📋 GRADUATION RATE PIVOT TABLE:")
pivot_table = df.pivot_table(
    values='GRADUADO',
    index='TIPO DE FORMACIÓN',
    columns='GÉNERO',
    aggfunc=lambda x: (x == 'SI').sum() / len(x) * 100
)
print(pivot_table.round(2))

In [ ]:
# Group by ESTRATO and GRADUADO to see patterns
print("📈 CROSS-TABULATION: ESTRATO × GRADUATION STATUS")
print("="*80)

# Create crosstab
crosstab_result = pd.crosstab(
    df['ESTRATO_CATEGORY'],
    df['GRADUADO'],
    margins=True,
    margins_name='Total'
)

print("Count Distribution:")
print(crosstab_result)

# Calculate percentages
print("\nPercentage Distribution (by row):")
crosstab_pct = pd.crosstab(
    df['ESTRATO_CATEGORY'],
    df['GRADUADO'],
    normalize='index'
) * 100
print(crosstab_pct.round(2))

---
## Part 6: Advanced Filtering and Analysis 🎯

### Task 6.1: Complex Conditions

In [ ]:
# Find female students from ESTRATO 1 or 2 who graduated
print("🎯 COMPLEX FILTERING: MULTIPLE CONDITIONS")
print("="*80)

complex_filter = df[
    (df['GÉNERO'] == 'FEMENINO') &
    ((df['ESTRATO_NUMERIC'] == 1) | (df['ESTRATO_NUMERIC'] == 2)) &
    (df['GRADUADO'] == 'SI')
]

print(f"Female students from ESTRATO 1-2 who graduated: {len(complex_filter):,}")
print(f"\nTop 5 programs for this group:")
print(complex_filter['PROGRAMA CURSADO'].value_counts().head())

print(f"\nTop 5 universities for this group:")
print(complex_filter['UNIVERSIDAD'].value_counts().head())

In [ ]:
# Use isin() for multiple value filtering
print("🔍 FILTERING WITH ISIN():")
print("="*80)

# Define target universities
top_universities = df['UNIVERSIDAD'].value_counts().head(5).index.tolist()

# Filter students from top universities
top_uni_students = df[df['UNIVERSIDAD'].isin(top_universities)]

print(f"Students from top 5 universities: {len(top_uni_students):,}")
print(f"Percentage of total: {(len(top_uni_students) / len(df)) * 100:.1f}%")

# Analyze graduation rates for top universities
print("\n🎓 Graduation Rates for Top 5 Universities:")
for uni in top_universities:
    uni_data = df[df['UNIVERSIDAD'] == uni]
    grad_rate = (uni_data['GRADUADO'] == 'SI').sum() / len(uni_data) * 100
    print(f"   {uni[:40]:<40} {grad_rate:.2f}%")

### Task 6.2: Top and Bottom Analysis

In [ ]:
# Find top 5 universities by graduation rate (minimum 50 students)
print("🏆 TOP PERFORMING UNIVERSITIES:")
print("="*80)

# Calculate graduation rates by university
university_stats = df.groupby('UNIVERSIDAD').agg({
    'GRADUADO': ['count', lambda x: (x == 'SI').sum()]
})

university_stats.columns = ['Total_Students', 'Graduated']
university_stats['Graduation_Rate'] = (
    university_stats['Graduated'] / university_stats['Total_Students']
) * 100

# Filter universities with at least 50 students
qualified_universities = university_stats[university_stats['Total_Students'] >= 50]

# Get top 5
top_5_universities = qualified_universities.nlargest(5, 'Graduation_Rate')

print("Top 5 Universities by Graduation Rate (min. 50 students):")
for idx, row in top_5_universities.iterrows():
    print(f"   {idx[:40]:<40} {row['Graduation_Rate']:.2f}% ({row['Total_Students']:.0f} students)")

In [ ]:
# Identify programs with lowest graduation rates
print("⚠️ PROGRAMS WITH LOWEST GRADUATION RATES:")
print("="*80)

# Calculate graduation rates by program
program_stats = df.groupby('PROGRAMA CURSADO').agg({
    'GRADUADO': ['count', lambda x: (x == 'SI').sum()]
})

program_stats.columns = ['Total_Students', 'Graduated']
program_stats['Graduation_Rate'] = (
    program_stats['Graduated'] / program_stats['Total_Students']
) * 100

# Filter programs with at least 30 students
qualified_programs = program_stats[program_stats['Total_Students'] >= 30]

# Get bottom 5
bottom_5_programs = qualified_programs.nsmallest(5, 'Graduation_Rate')

print("Bottom 5 Programs by Graduation Rate (min. 30 students):")
for idx, row in bottom_5_programs.iterrows():
    print(f"   {idx[:40]:<40} {row['Graduation_Rate']:.2f}% ({row['Total_Students']:.0f} students)")

---
## Part 7: Insights and Conclusions 💡

### Task 7.1: Key Metrics Summary

In [ ]:
print("📊 KEY PERFORMANCE INDICATORS SUMMARY")
print("="*80)

# Overall metrics
total_beneficiaries = len(df)
overall_graduation_rate = (df['GRADUADO'] == 'SI').sum() / len(df) * 100

# Calculate average students per year
years_covered = df['CONVOCATORIA'].nunique()
avg_students_per_year = total_beneficiaries / years_covered

# Gender metrics
female_percentage = (df['GÉNERO'] == 'FEMENINO').sum() / len(df) * 100
female_grad_rate = (df[df['GÉNERO'] == 'FEMENINO']['GRADUADO'] == 'SI').sum() / (df['GÉNERO'] == 'FEMENINO').sum() * 100
male_grad_rate = (df[df['GÉNERO'] == 'MASCULINO']['GRADUADO'] == 'SI').sum() / (df['GÉNERO'] == 'MASCULINO').sum() * 100
gender_gap = abs(female_grad_rate - male_grad_rate)

print("\n📈 OVERALL PROGRAM METRICS:")
print(f"   • Total beneficiaries: {total_beneficiaries:,}")
print(f"   • Years covered: {years_covered}")
print(f"   • Average students per year: {avg_students_per_year:,.0f}")
print(f"   • Overall graduation rate: {overall_graduation_rate:.2f}%")

print("\n👥 GENDER METRICS:")
print(f"   • Female representation: {female_percentage:.1f}%")
print(f"   • Female graduation rate: {female_grad_rate:.2f}%")
print(f"   • Male graduation rate: {male_grad_rate:.2f}%")
print(f"   • Gender gap in graduation: {gender_gap:.2f} percentage points")

# Most common profile
print("\n👤 MOST COMMON BENEFICIARY PROFILE:")
most_common_gender = df['GÉNERO'].mode()[0]
most_common_estrato = df['ESTRATO_NUMERIC'].mode()[0]
most_common_education = df['TIPO DE FORMACIÓN'].mode()[0]
most_common_subregion = df['SUBREGIÓN DE RESIDENCIA'].mode()[0]

print(f"   • Gender: {most_common_gender}")
print(f"   • Socioeconomic stratum: {most_common_estrato:.0f}")
print(f"   • Education type: {most_common_education}")
print(f"   • Subregion: {most_common_subregion}")

### Task 7.2: Program Effectiveness Analysis

In [ ]:
print("🎯 PROGRAM EFFECTIVENESS BY SOCIOECONOMIC SEGMENT")
print("="*80)

# Compare graduation rates by socioeconomic groups
estrato_groups = {
    'Low (1-2)': df[df['ESTRATO_CATEGORY'] == 'Low'],
    'Medium (3-4)': df[df['ESTRATO_CATEGORY'] == 'Medium'],
    'High (5-6)': df[df['ESTRATO_CATEGORY'] == 'High']
}

print("\n📊 GRADUATION RATES BY SOCIOECONOMIC GROUP:")
for group_name, group_data in estrato_groups.items():
    if len(group_data) > 0:
        total = len(group_data)
        graduated = (group_data['GRADUADO'] == 'SI').sum()
        rate = (graduated / total) * 100
        print(f"   {group_name:<15} {rate:>6.2f}% ({graduated:,}/{total:,} students)")

# Education type effectiveness
print("\n📚 GRADUATION RATES BY EDUCATION TYPE:")
education_effectiveness = df.groupby('TIPO DE FORMACIÓN').apply(
    lambda x: (x['GRADUADO'] == 'SI').sum() / len(x) * 100
).sort_values(ascending=False)

for edu_type, rate in education_effectiveness.items():
    student_count = (df['TIPO DE FORMACIÓN'] == edu_type).sum()
    print(f"   {edu_type:<25} {rate:>6.2f}% ({student_count:,} students)")

# Regional performance
print("\n🌍 TOP 3 REGIONS BY PERFORMANCE:")
region_performance = df.groupby('SUBREGIÓN DE RESIDENCIA').apply(
    lambda x: (x['GRADUADO'] == 'SI').sum() / len(x) * 100
).nlargest(3)

for i, (region, rate) in enumerate(region_performance.items(), 1):
    student_count = (df['SUBREGIÓN DE RESIDENCIA'] == region).sum()
    print(f"   {i}. {region:<20} {rate:>6.2f}% ({student_count:,} students)")

### Task 7.3: Export Results

In [ ]:
print("💾 EXPORTING ANALYSIS RESULTS")
print("="*80)

# Create summary DataFrame with key findings
summary_metrics = {
    'Metric': [
        'Total Beneficiaries',
        'Overall Graduation Rate (%)',
        'Female Graduation Rate (%)',
        'Male Graduation Rate (%)',
        'Gender Gap (pp)',
        'Low Estrato Graduation Rate (%)',
        'Medium Estrato Graduation Rate (%)',
        'High Estrato Graduation Rate (%)'
    ],
    'Value': [
        total_beneficiaries,
        overall_graduation_rate,
        female_grad_rate,
        male_grad_rate,
        gender_gap,
        (df[df['ESTRATO_CATEGORY'] == 'Low']['GRADUADO'] == 'SI').sum() / len(df[df['ESTRATO_CATEGORY'] == 'Low']) * 100 if len(df[df['ESTRATO_CATEGORY'] == 'Low']) > 0 else 0,
        (df[df['ESTRATO_CATEGORY'] == 'Medium']['GRADUADO'] == 'SI').sum() / len(df[df['ESTRATO_CATEGORY'] == 'Medium']) * 100 if len(df[df['ESTRATO_CATEGORY'] == 'Medium']) > 0 else 0,
        (df[df['ESTRATO_CATEGORY'] == 'High']['GRADUADO'] == 'SI').sum() / len(df[df['ESTRATO_CATEGORY'] == 'High']) * 100 if len(df[df['ESTRATO_CATEGORY'] == 'High']) > 0 else 0
    ]
}

summary_df = pd.DataFrame(summary_metrics)

# Export summary
summary_df.to_csv('scholarship_analysis_summary.csv', index=False)
print("✅ Summary metrics exported to 'scholarship_analysis_summary.csv'")

# Export top performing universities
top_5_universities.to_csv('top_performing_universities.csv')
print("✅ Top performing universities exported to 'top_performing_universities.csv'")

# Export graduation rates by category
graduation_by_category = df.groupby(['GÉNERO', 'TIPO DE FORMACIÓN', 'ESTRATO_CATEGORY']).agg({
    'GRADUADO': ['count', lambda x: (x == 'SI').sum(), lambda x: (x == 'SI').sum() / len(x) * 100]
})
graduation_by_category.columns = ['Total', 'Graduated', 'Graduation_Rate(%)']
graduation_by_category.to_csv('graduation_rates_detailed.csv')
print("✅ Detailed graduation rates exported to 'graduation_rates_detailed.csv'")

print("\n📊 Analysis complete! All results have been exported.")

---
## 🎉 Workshop Complete!

### 📝 Key Takeaways:

1. **Data Quality**: Real-world data requires cleaning and type conversion
2. **Gender Insights**: Analyzed gender gaps in education access and success
3. **Socioeconomic Factors**: Lower strata students face different challenges
4. **Program Effectiveness**: Different education types show varying success rates
5. **Regional Patterns**: Geographic differences in program outcomes

### 💡 Skills Practiced:
- Data loading and exploration (The Big 3)
- Data cleaning and type conversion
- Pandas filtering and selection (loc/iloc)
- NumPy arrays and vectorized operations
- GroupBy operations and aggregations
- Complex filtering and analysis
- Exporting results for reporting

### 🚀 Next Steps:
- Try the challenge questions from the README
- Explore additional patterns in the data
- Create visualizations (when you learn them)
- Share your insights with the class!